In [ ]:
# |default_exp mongo_episode

In [ ]:
from mongo import print_logs

print_logs(5)

{'_id': ObjectId('67b2080dcd129af4715c4e0a'), 'operation': 'update', 'entite': 'episodes', 'desc': '18 déc. 1958 00:00 - Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958', 'date': datetime.datetime(2025, 2, 16, 15, 45, 17, 940000)}
{'_id': ObjectId('67b207d5cd129af4715c4e00'), 'operation': 'delete', 'entite': 'episodes', 'desc': '16 Feb 2025 16:44 - test RSS 1', 'date': datetime.datetime(2025, 2, 16, 15, 44, 21, 497000)}
{'_id': ObjectId('67b207d5cd129af4715c4dfd'), 'operation': 'insert', 'entite': 'episodes', 'desc': '16 Feb 2025 16:44 - test RSS 1', 'date': datetime.datetime(2025, 2, 16, 15, 44, 21, 490000)}
{'_id': ObjectId('67b207bbcd129af4715c4df4'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '12 Jan 2025 08:59 - Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers... -> 12 Jan 2025 08:59', 'date': datetime.datetime(2025, 2, 16, 15, 43, 55, 851000)}
{'_id': ObjectId('67b0e319a5aa5d490305d913'), 


# Episode entity

In [ ]:
# |export

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# from datasets import load_dataset

# Import dbus only if available (not available in Docker)
try:
    import dbus

    DBUS_AVAILABLE = True
except ImportError:
    DBUS_AVAILABLE = False

from functools import wraps
from typing import Callable, Any


def prevent_sleep(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Decorator that prevents the system from sleeping during a long-running process.

    Connects to the D-Bus session bus and inhibits the screensaver, ensuring
    that the system does not enter sleep mode while the decorated function runs.

    Note: Only works if dbus-python is installed. In Docker environments without dbus,
    this decorator does nothing and simply calls the function.

    Args:
        func (Callable[..., Any]): The function to be decorated.

    Returns:
        Callable[..., Any]: The wrapped function that prevents system sleep
        during its execution.

    Example:
        @prevent_sleep
        def long_task():
            ...
    """

    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        """
        Wrapper function that manages sleep inhibition.

        Args:
            *args: Variable length argument list for the decorated function.
            **kwargs: Arbitrary keyword arguments for the decorated function.

        Returns:
            Any: The result of executing the decorated function.
        """
        if not DBUS_AVAILABLE:
            # D-Bus not available (e.g., in Docker), just run the function
            return func(*args, **kwargs)

        # Connect to the D-Bus session bus and obtain the screensaver interface.
        bus = dbus.SessionBus()
        proxy = bus.get_object(
            "org.freedesktop.ScreenSaver", "/org/freedesktop/ScreenSaver"
        )
        interface = dbus.Interface(proxy, "org.freedesktop.ScreenSaver")

        # Inhibit sleep mode.
        cookie = interface.Inhibit("my_script", "Long running process")
        print("Mise en veille désactivée")

        try:
            result = func(*args, **kwargs)
            return result
        finally:
            # Re-enable normal sleep mode.
            interface.UnInhibit(cookie)
            print("Mise en veille normale réactivée")

    return wrapper


# @prevent_sleep
def extract_whisper(mp3_filename: str) -> str:
    """
    Extract transcription text from an audio file using a Whisper model.

    Loads a Whisper model and its processor from Hugging Face, sets up an automatic
    speech recognition pipeline, and processes the provided MP3 file to return
    the transcription text.

    Args:
        mp3_filename (str): Path to the MP3 audio file.

    Returns:
        str: Transcribed text extracted from the audio.

    Example:
        >>> transcription = extract_whisper("path/to/audio.mp3")
    """
    device: str = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id: str = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )

    # Load a sample dataset (this sample is loaded for demonstration purposes and is not used in transcription).
    # dataset = load_dataset(
    #     "distil-whisper/librispeech_long", "clean", split="validation"
    # )
    # sample = dataset[0]["audio"]

    result = pipe(
        mp3_filename,
        return_timestamps=True,
    )

    return result["text"]

In [ ]:
# |export

from bson import ObjectId
from mongo import get_collection, get_DB_VARS, mongolog
from datetime import datetime
import requests
from typing import Dict, List, Optional, Union
from llm import get_azure_llm
from llama_index.core.llms import ChatMessage
import json
import os
from config import get_audio_path, AUDIO_PATH


DATE_FORMAT: str = "%Y-%m-%dT%H:%M:%S"
LOG_DATE_FORMAT: str = "%d %b %Y %H:%M"


class Episode:
    def __init__(
        self, date: str, titre: str, collection_name: str = "episodes"
    ) -> None:
        """Initialise une instance d'Episode.

        Args:
            date (str): La date de l'épisode au format "2024-12-22T09:59:39" conforme à DATE_FORMAT.
            titre (str): Le titre de l'épisode.
            collection_name (str, optional): Le nom de la collection dans la base de données. Défaut: "episodes".

        Notes:
            Si l'épisode existe déjà en base, ses attributs seront chargés.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        self.date: datetime = Episode.get_date_from_string(date)
        self.titre: str = titre

        if self.exists():
            episode = self.collection.find_one({"titre": self.titre, "date": self.date})
            self.description: Optional[str] = episode.get("description")
            self.url_telechargement: Optional[str] = episode.get("url")
            self.audio_rel_filename: Optional[str] = episode.get("audio_rel_filename")
            self.transcription: Optional[str] = episode.get("transcription")
            self.type: Optional[str] = episode.get("type")
            self.duree: int = episode.get("duree", -1)
        else:
            self.description = None
            self.url_telechargement = None
            self.audio_rel_filename = None
            self.transcription = None
            self.type = None
            self.duree = -1  # en secondes

    @classmethod
    def from_oid(cls, oid: ObjectId, collection_name: str = "episodes") -> "Episode":
        """Crée un épisode à partir d'un ObjectId dans la base de données.

        Args:
            oid (ObjectId): L'identifiant de l'épisode dans Mongo.
            collection_name (str, optional): Le nom de la collection. Défaut: "episodes".

        Returns:
            Episode: L'instance d'Episode correspondante.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        document = collection.find_one({"_id": oid})
        date_doc_str = cls.get_string_from_date(document.get("date"), DATE_FORMAT)
        instance = cls(date=date_doc_str, titre=document.get("titre"))
        return instance

    @classmethod
    def from_date(
        cls, date: datetime, collection_name: str = "episodes"
    ) -> Optional["Episode"]:
        """Crée un épisode à partir d'une date dans la base de données.

        Args:
            date (datetime): La date recherchée.
            collection_name (str, optional): Le nom de la collection. Défaut: "episodes".

        Returns:
            Optional[Episode]: L'instance d'Episode si trouvée, sinon None.
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        start_date = datetime(date.year, date.month, date.day)
        end_date = datetime(date.year, date.month, date.day, 23, 59, 59)
        document = collection.find_one({"date": {"$gte": start_date, "$lte": end_date}})
        if document:
            date_doc_str = cls.get_string_from_date(document.get("date"), DATE_FORMAT)
            instance = cls(date=date_doc_str, titre=document.get("titre"))
            return instance
        else:
            return None

    def exists(self) -> bool:
        """Vérifie si l'épisode existe dans la base de données.

        Returns:
            bool: True si l'épisode existe, False sinon.
        """
        return (
            self.collection.find_one({"titre": self.titre, "date": self.date})
            is not None
        )

    def keep(self) -> int:
        """Télécharge le fichier audio si nécessaire et conserve l'épisode dans la base de données.

        Returns:
            int: 1 si une nouvelle entrée est créée en base, 0 sinon.
        """
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        if not self.exists():
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} nouveau: Duree: {self.duree}, Type: {self.type}"
            )
            mongolog("insert", self.collection.name, message_log)
            self.download_audio(verbose=True)
            self.collection.insert_one(
                {
                    "titre": self.titre,
                    "date": self.date,
                    "description": self.description,
                    "url": self.url_telechargement,
                    "audio_rel_filename": self.audio_rel_filename,
                    "transcription": self.transcription,
                    "type": self.type,
                    "duree": self.duree,
                }
            )
            return 1
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} deja existant"
            )
            mongolog("update", self.collection.name, message_log)
            return 0

    def update_date(self, new_date: datetime) -> None:
        """Met à jour la date de l'épisode dans la base de données.

        Args:
            new_date (datetime): La nouvelle date de l'épisode.
        """
        self.collection.update_one(
            {"_id": self.get_oid()}, {"$set": {"date": new_date}}
        )
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre} -> {Episode.get_string_from_date(new_date, format=LOG_DATE_FORMAT)}"
        self.date = new_date
        mongolog("force_update", self.collection.name, message_log)

    def remove(self) -> None:
        """Supprime l'épisode de la base de données."""
        message_log = f"{Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} - {self.titre}"
        self.collection.delete_one({"titre": self.titre, "date": self.date})
        mongolog("delete", self.collection.name, message_log)

    def get_oid(self) -> Optional[ObjectId]:
        """Récupère l'identifiant Mongo (_id) de l'épisode.

        Returns:
            Optional[ObjectId]: L'ObjectId de l'épisode s'il existe, sinon None.
        """
        document = self.collection.find_one({"titre": self.titre, "date": self.date})
        if document:
            return document["_id"]
        else:
            return None

    @staticmethod
    def get_date_from_string(date: str, DATE_FORMAT: str = DATE_FORMAT) -> datetime:
        """Convertit une chaîne de caractères en objet datetime.

        Args:
            date (str): La chaîne représentant la date.
            DATE_FORMAT (str, optional): Le format de la date. Défaut est DATE_FORMAT.

        Returns:
            datetime: L'objet datetime correspondant.
        """
        return datetime.strptime(date, DATE_FORMAT)

    @staticmethod
    def get_string_from_date(date: datetime, format: Optional[str] = None) -> str:
        """Convertit un objet datetime en chaîne de caractères.

        Args:
            date (datetime): L'objet datetime.
            format (Optional[str], optional): Le format de sortie. Si None, DATE_FORMAT est utilisé.

        Returns:
            str: La chaîne représentant la date.
        """
        if format is not None:
            return date.strftime(format)
        else:
            return date.strftime(DATE_FORMAT)

    @staticmethod
    def format_duration(seconds: int) -> str:
        """Convertit une durée en secondes au format HH:MM:SS.

        Args:
            seconds (int): La durée en secondes.

        Returns:
            str: La durée formatée en chaîne de caractères.
        """
        if seconds < 0:
            return f"-{Episode.format_duration(-seconds)}"
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = seconds % 60
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    def __str__(self) -> str:
        """Renvoie une représentation textuelle de l'épisode.

        Returns:
            str: Les informations de l'épisode sous forme de chaîne de caractères.
        """
        return (
            f"_oid: {self.get_oid()}\n"
            f"Date: {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)}\n"
            f"Titre: {self.titre}\n"
            f"Description: {self.description}\n"
            f"URL de téléchargement: {self.url_telechargement}\n"
            f"Fichier audio: {self.audio_rel_filename}\n"
            f"Duree: {self.duree} en secondes ({Episode.format_duration(self.duree)})\n"
            f"Transcription: {self.transcription[:100] if self.transcription else 'No transcription yet available'}..."
        )

    def __repr__(self) -> str:
        """Renvoie une représentation officielle de l'objet.

        Returns:
            str: La représentation de l'objet (équivalente à __str__).
        """
        return self.__str__()

    def download_audio(self, verbose: bool = False) -> None:
        """Télécharge le fichier audio à partir de l'URL de téléchargement et le sauvegarde localement.

        Args:
            verbose (bool, optional): Si True, affiche des messages d'information. Défaut False.
        """
        if self.url_telechargement is None:
            return
        year = str(self.date.year)
        full_audio_path = get_audio_path(AUDIO_PATH, year)
        full_filename = os.path.join(
            full_audio_path, os.path.basename(self.url_telechargement)
        )
        self.audio_rel_filename = os.path.relpath(
            full_filename, get_audio_path(AUDIO_PATH, year="")
        )
        if not os.path.exists(full_filename):
            if verbose:
                print(
                    f"Téléchargement de {self.url_telechargement} vers {full_filename}"
                )
            response = requests.get(self.url_telechargement)
            with open(full_filename, "wb") as file:
                file.write(response.content)
        else:
            if verbose:
                print(f"Le fichier {full_filename} existe déjà. Ignoré.")

    def set_transcription(self, verbose: bool = False, keep_cache: bool = True) -> None:
        """Extrait et sauvegarde la transcription de l'audio en utilisant un modèle Whisper.

        Utilise le cache si disponible ou extrait la transcription de l'audio.

        Args:
            verbose (bool, optional): Si True, affiche des messages d'information. Défaut False.
            keep_cache (bool, optional): Si True, sauvegarde la transcription dans un fichier cache. Défaut True.
        """
        if self.transcription is not None:
            if verbose:
                print("Transcription existe deja")
            return
        mp3_fullfilename = get_audio_path(AUDIO_PATH, year="") + self.audio_rel_filename
        cache_transcription_filename = f"{os.path.splitext(mp3_fullfilename)[0]}.txt"
        if os.path.exists(cache_transcription_filename):
            if verbose:
                print(f"Transcription cachee trouvee: {cache_transcription_filename}")
            with open(cache_transcription_filename, "r") as file:
                self.transcription = file.read()
            self.collection.update_one(
                {"_id": self.get_oid()},
                {"$set": {"transcription": self.transcription}},
            )
            return

        self.transcription = extract_whisper(mp3_fullfilename)
        if keep_cache:
            with open(cache_transcription_filename, "w") as f:
                f.write(self.transcription)
        self.collection.update_one(
            {"_id": self.get_oid()}, {"$set": {"transcription": self.transcription}}
        )

    def to_dict(self) -> Dict[str, Union[str, datetime, int, None]]:
        """Convertit l'épisode en dictionnaire.

        Returns:
            Dict[str, Union[str, datetime, int, None]]: Dictionnaire contenant les informations de l'épisode.
                Les clés sont ['date', 'titre', 'description', 'url_telechargement', 'audio_rel_filename', 'transcription', 'type', 'duree'].
        """
        return {
            "date": self.date,
            "titre": self.titre,
            "description": self.description,
            "url_telechargement": self.url_telechargement,
            "audio_rel_filename": self.audio_rel_filename,
            "transcription": self.transcription,
            "type": self.type,
            "duree": self.duree,
        }

    def get_all_auteurs(self) -> List[str]:
        """Extrait la liste de tous les auteurs mentionnés dans la transcription.

        Notes:
            Utilise le modèle GPT-4 via Azure LLM pour extraire une liste JSON de noms d'auteurs.

        Returns:
            List[str]: La liste des auteurs détectés.
        """
        if self.transcription is None:
            return []

        llm_structured_output = get_azure_llm("gpt-4o")
        response_schema = {
            "type": "json_schema",
            "json_schema": {
                "name": "AuthorList",
                "schema": {
                    "type": "object",
                    "properties": {
                        "Authors": {
                            "type": "array",
                            "items": {
                                "type": "string",
                                "description": "Une liste des auteurs extraits de la transcription",
                            },
                        }
                    },
                    "required": ["Authors"],
                    "additionalProperties": False,
                },
            },
        }
        response = llm_structured_output.chat(
            messages=[
                ChatMessage(
                    role="system",
                    content="Tu es un assistant utile qui retourne une liste JSON de noms d'auteurs.",
                ),
                ChatMessage(
                    role="user",
                    content=f"Est-ce que tu peux me lister tous les noms d'auteurs dont on parle des oeuvres \
dans cette transcription d'un épisode du masque et la plume \
diffuse le {self.date.strftime('%d %b %Y')}. \
Je veux toujours avoir le prénom et le nom complet de chaque auteur. \
Voici cette transcription : {self.transcription} ",
                ),
            ],
            response_format=response_schema,
        )
        try:
            json_dict = json.loads(response.message.content)
        except json.JSONDecodeError as e:
            print("Error parsing JSON:", e)
            print("Raw response:", response.message.content)
            return []
        return json_dict["Authors"]

In [ ]:
episode = Episode(
    date="2025-01-12T08:59:39",
    titre="Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...",
)
episode

_oid: 678ad2ff9010ec6dc606dce9
Date: 12 Jan 2025 08:59
Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-22.mp3
Fichier audio: 2

In [ ]:
episode.update_date(datetime(2025, 1, 12, 8, 59, 39))
episode

_oid: 678ad2ff9010ec6dc606dce9
Date: 12 Jan 2025 08:59
Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-22.mp3
Fichier audio: 2

In [ ]:
episode.to_dict()

{'date': datetime.datetime(2025, 1, 12, 8, 59, 39),
 'titre': 'Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...',
 'description': "durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface\xa0; une cité magique abrite un amour perdu\xa0; un cinéaste entre un tournage en Afrique et la chute d'un homme nu\xa0; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé\xa0; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault",
 'url_telechargement': 'https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-2

In [ ]:
episode = Episode.from_oid(ObjectId("678586de9ff0dfcda11eacf8"))
episode

_oid: 678586de9ff0dfcda11eacf8
Date: 22 Dec 2024 09:59
Titre: Les nouvelles pages de Marc Dugain, Emmanuelle Lambert, Emil Ferris, Fabrice Caro et Mathieu Palain
Description: durée : 00:47:52 - Le Masque et la Plume - par : Rebecca Manzoni - Il est encore temps d'ajouter quelques livres sous le sapin - invités : Raphaelle Leyris, Jean-Marc Proust, Patricia Martin, Laurent CHALUMEAU - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/7e653bf4-87a5-42f4-864b-9208e206a295/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3
Fichier audio: 2024/14007-22.12.2024-ITEMA_23973143-2024F4007S0357-22.mp3
Duree: 2872 en secondes (00:47:52)
Transcription:  France Inter Le masque et la plume Cinq romans s

In [ ]:
episode = Episode.from_date(datetime(2025, 1, 12))
episode

_oid: 678ad2ff9010ec6dc606dce9
Date: 12 Jan 2025 08:59
Titre: Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers...
Description: durée : 00:48:41 - Le Masque et la Plume - par : Rebecca Manzoni - Un passé nazi qui refait surface ; une cité magique abrite un amour perdu ; un cinéaste entre un tournage en Afrique et la chute d'un homme nu ; un danseur offre à son fils adoptif la quête d'un héritage familial mouvementé ; un jeune père se confronte à son propre passé douloureux. - invités : Raphaelle Leyris, Hubert ARTUS, Jean-Marc Proust, Elisabeth Philippe - Raphaëlle Leyris : Journaliste au Monde, critique littéraire, Hubert Artus : Journaliste et chroniqueur littéraire, Jean-Marc Proust : Auteur et critique (Slate), Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-22.mp3
Fichier audio: 2

In [ ]:
epis_26jan = Episode.from_date(datetime(2025, 1, 26))
epis_26jan.get_all_auteurs()

['Pierre Lemaitre',
 'Leila Slimani',
 'Nathalie Azoulay',
 'Jeanne Rivière',
 'Milena Agus',
 'Haruki Murakami',
 'Constantin Alexandrakis',
 'Vanessa Springora',
 'Neige Sinault',
 'Jean Echenoz',
 'Chimamanda Ngozi Adichie',
 'Johann Sfar',
 'Guillaume Lebrun',
 'Jean-Patrick Manchette']

# RSS_episode

In [ ]:
# |export

from feedparser.util import FeedParserDict
from transformers import pipeline
import locale
from datetime import datetime

RSS_DUREE_MINI_MINUTES: int = 15
RSS_DATE_FORMAT: str = (
    "%a, %d %b %Y %H:%M:%S %z"  # Exemple: "Sun, 29 Dec 2024 10:59:39 +0100"
)


class RSS_episode(Episode):
    def __init__(self, date: str, titre: str) -> None:
        """
        Initialize an RSS_episode instance.

        Args:
            date (str): The episode date in the format "2024-12-22T09:59:39".
            titre (str): The title of the episode.
        """
        super().__init__(date, titre)

    @classmethod
    def from_feed_entry(cls, feed_entry: FeedParserDict) -> "RSS_episode":
        """
        Create an RSS_episode instance from an RSS feed entry.

        Args:
            feed_entry (FeedParserDict): The entry from the RSS feed.

        Returns:
            RSS_episode: The created RSS_episode instance.
        """
        locale.setlocale(locale.LC_TIME, "en_US.UTF-8")
        date_rss: datetime = datetime.strptime(feed_entry.published, RSS_DATE_FORMAT)
        date_rss_str: str = cls.get_string_from_date(date_rss, DATE_FORMAT)
        inst = cls(
            date=date_rss_str,
            titre=feed_entry.title,
        )
        inst.description = feed_entry.summary

        for link in feed_entry.links:
            if link.type == "audio/mpeg":
                inst.url_telechargement = link.href
                break

        inst.type = cls.set_titre(inst.titre + " " + inst.description)
        inst.duree = cls.get_duree_in_seconds(feed_entry.itunes_duration)  # in seconds

        return inst

    @staticmethod
    def get_duree_in_seconds(duree: str) -> int:
        """
        Convert a duration string into total seconds.

        The duration can be in formats "HH:MM:SS", "HH:MM", or simply seconds.

        Args:
            duree (str): The duration as a string.

        Returns:
            int: The duration expressed in total seconds.
        """
        duree_parts = duree.split(":")
        if len(duree_parts) == 3:
            return (
                int(duree_parts[0]) * 3600
                + int(duree_parts[1]) * 60
                + int(duree_parts[2])
            )
        elif len(duree_parts) == 2:
            return int(duree_parts[0]) * 60 + int(duree_parts[1])
        else:
            return int(duree_parts[0])

    def keep(self) -> int:
        """
        Save the episode to the database if conditions are met.

        The episode is saved if:
            - The duration is greater than RSS_DUREE_MINI_MINUTES * 60 seconds.
            - The type is equal to "livres".

        Returns:
            int: 1 if an entry is created in the database, 0 otherwise.
        """
        if (self.duree > RSS_DUREE_MINI_MINUTES * 60) and (self.type == "livres"):
            return super().keep()
        else:
            print(
                f"Episode du {Episode.get_string_from_date(self.date, format=LOG_DATE_FORMAT)} ignored: Duree: {self.duree}, Type: {self.type}"
            )
            return 0

    @staticmethod
    def set_titre(description: str) -> str:
        """
        Classify the episode by using a zero-shot classification model from HuggingFace based on the provided description.

        Args:
            description (str): The description combining the title and summary.

        Returns:
            str: The label with the highest score among ["livres", "films", "pièces de théâtre"].
        """
        classifier = pipeline(
            "zero-shot-classification", model="facebook/bart-large-mnli"
        )
        labels = ["livres", "films", "pièces de théâtre"]

        result = classifier(description, labels)
        return result["labels"][0]

In [ ]:
from datetime import datetime
import pytz

now = datetime.now(tz=pytz.timezone("Europe/Paris"))

rss1 = RSS_episode(RSS_episode.get_string_from_date(now), "test RSS 1")
print(f"Est-ce que l episode existe ? {rss1.exists()}")

rss1.duree = 4000
rss1.type = "livres"
rss1.keep()
print(f"et maintenant, st-ce que rss1 existe ? {rss1.exists()}")
print(f"et voici l'id de rss1 : {rss1.get_oid()}")

rss1.remove()
print(f"après nettoyage, est-ce que rss1 existe ? {rss1.exists()}")
print(f"et son oid : {rss1.get_oid()}")

Est-ce que l episode existe ? False
Episode du 16 Feb 2025 18:04 nouveau: Duree: 4000, Type: livres
et maintenant, st-ce que rss1 existe ? True
et voici l'id de rss1 : 67b21aa981c54d6fac1f266f
après nettoyage, est-ce que rss1 existe ? False
et son oid : None


In [ ]:
import feedparser
from config import get_RSS_URL

parsed_flow = feedparser.parse(get_RSS_URL())

rss2 = RSS_episode.from_feed_entry(parsed_flow.entries[0])
rss2

Device set to use cpu


_oid: None
Date: 16 Feb 2025 10:59
Titre: Faut-il aller voir "The Brutalist", "Maria", "La Pampa", "Bridget Jones 4" et "La Mer au Loin" ?
Description: durée : 00:53:30 - Le Masque et la Plume - par : Rebecca Manzoni - Un architecte rescapé de la Shoah en Amérique exprime son génie créatif ; les derniers jours de Maria Callas à Paris ; motocross, ambition et secret entre un jeune pilote et son mécano dans un village ; le retour de Bridget Jones ; un exil nord-africain à Marseille, entre espoir et musique. - invités : Xavier Leherpeur, Charlotte LIPINSKA, Ariane Allard, Nicolas SCHALLER - Xavier Leherpeur : Chroniqueur et critique de cinéma (7e Obsession), Charlotte Lipinska : Critique française de cinéma, Ariane Allard : Critique de cinéma pour le magazine Positif, Nicolas Schaller : Journaliste pour L'Obs - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/cf02d19c-09db-4672-a20f-715d41890b3c/14007-16.02.2025-ITEMA_24030701-2025F4007S0047-NET_MFI_EF7DD9CE

In [ ]:
rss2.keep()
rss2.exists()

Episode du 16 Feb 2025 10:59 ignored: Duree: 3210, Type: films


False

In [ ]:
# rss2.remove()

In [ ]:
print_logs(5)

{'_id': ObjectId('67b21aa981c54d6fac1f2671'), 'operation': 'delete', 'entite': 'episodes', 'desc': '16 Feb 2025 18:04 - test RSS 1', 'date': datetime.datetime(2025, 2, 16, 17, 4, 41, 328000)}
{'_id': ObjectId('67b21aa981c54d6fac1f266e'), 'operation': 'insert', 'entite': 'episodes', 'desc': '16 Feb 2025 18:04 - test RSS 1', 'date': datetime.datetime(2025, 2, 16, 17, 4, 41, 324000)}
{'_id': ObjectId('67b21aa681c54d6fac1f2665'), 'operation': 'force_update', 'entite': 'episodes', 'desc': '12 Jan 2025 08:59 - Les nouvelles pages de Vanessa Springora, Haruki Murakami, Jean Echenoz, Amanda Sthers... -> 12 Jan 2025 08:59', 'date': datetime.datetime(2025, 2, 16, 17, 4, 38, 862000)}
{'_id': ObjectId('67b2080dcd129af4715c4e0a'), 'operation': 'update', 'entite': 'episodes', 'desc': '18 déc. 1958 00:00 - Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958', 'date': datetime.datetime(2025, 2, 16, 15, 45, 17, 940000)}
{'_id': ObjectId('67b207d5cd129af4715c4e00'), 'op

In [ ]:
episode3 = Episode(
    date="2024-11-10T09:59:39",
    titre="La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...",
)
episode3

_oid: 6773e32258fc5717f3516b9f
Date: 10 Nov 2024 09:59
Titre: La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...
Description: durée : 00:46:03 - Le Masque et la Plume - par : Rebecca Manzoni - En direct de la 42ᵉ édition de la Foire du livre de Brive-la-Gaillarde, nos critiques vous disent ce qu'ils ont pensé de "Mon assassin" de Daniel Pennac, "Le rêve du jaguar" de Miguel Bonnefoy, "Les guerriers de l’hiver" d’Olivier Norek, "La règle du crime" de Colson Whitehead...
 - invités : Arnaud Viviant, Elisabeth Philippe, Jean-Marc Proust, Patricia Martin - Arnaud Viviant : Critique littéraire (Revue Regards), Elisabeth Philippe : Critique littéraire (L'Obs), Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/f5779476-9d52-4bfb-a839-bf82751eaebb/14007-10.11.2024-ITEMA_23920

In [ ]:
episode4 = Episode.from_date(Episode.get_date_from_string("10/11/2024", "%d/%m/%Y"))
episode4

_oid: 6773e32258fc5717f3516b9f
Date: 10 Nov 2024 09:59
Titre: La foire du livre de Brive : les romans de Daniel Pennac, Colson Whitehead, Olivier Norek, Miguel Bonnefoy...
Description: durée : 00:46:03 - Le Masque et la Plume - par : Rebecca Manzoni - En direct de la 42ᵉ édition de la Foire du livre de Brive-la-Gaillarde, nos critiques vous disent ce qu'ils ont pensé de "Mon assassin" de Daniel Pennac, "Le rêve du jaguar" de Miguel Bonnefoy, "Les guerriers de l’hiver" d’Olivier Norek, "La règle du crime" de Colson Whitehead...
 - invités : Arnaud Viviant, Elisabeth Philippe, Jean-Marc Proust, Patricia Martin - Arnaud Viviant : Critique littéraire (Revue Regards), Elisabeth Philippe : Critique littéraire (L'Obs), Jean-Marc Proust : Auteur et critique (Slate), Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/f5779476-9d52-4bfb-a839-bf82751eaebb/14007-10.11.2024-ITEMA_23920

In [ ]:
episode4.transcription = None
episode4.set_transcription(verbose=True)

Transcription cachee trouvee: /workspaces/lmelp/audios/2024/14007-10.11.2024-ITEMA_23920569-2024F4007S0315-22.txt


In [ ]:
episode4.transcription[:100]

' France Inter Le masque et la plume Bienvenue à Brive la Gaillarde, où se tient la 42e foire du livr'

In [ ]:
episode_tres_court = Episode.from_date(
    Episode.get_date_from_string("27/09/1992", "%d/%m/%Y")
)
episode_tres_court.transcription = None

In [ ]:
episode_tres_court.audio_rel_filename

mp3_fullfilename = (
    get_audio_path(AUDIO_PATH, year="") + episode_tres_court.audio_rel_filename
)
cache_transcription_filename = f"{os.path.splitext(mp3_fullfilename)[0]}.txt"
cache_transcription_filename
# decommenter pour supprimer le fichier cache ce qui lancera le traitement whisper
# !rm {cache_transcription_filename}

'/workspaces/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.txt'

In [ ]:
episode_tres_court.set_transcription(verbose=True)

Transcription cachee trouvee: /workspaces/lmelp/audios/1992/14007-27.09.1992-ITEMA_23787897-2024F4007E0094-27.txt


# WEB_episode

In [ ]:
# |export

import requests
from bs4 import BeautifulSoup
import json
import locale
from datetime import datetime
from typing import Optional, Dict, Any

WEB_DATE_FORMAT: str = (
    "%d %b %Y"  # '26 août 2024', '20 oct. 2024', '22 sept. 2024', etc.
)


class WEB_episode(Episode):
    """Représente un épisode web avec ses attributs et méthodes de conversion et récupération des données."""

    def __init__(self, date: str, titre: str) -> None:
        """Initialise une instance de WEB_episode.

        Args:
            date (str): La date de l'épisode au format "2024-12-22T09:59:39".
            titre (str): Le titre de l'épisode.
        """
        super().__init__(date, titre)

    @staticmethod
    def parse_web_date(
        web_date: str, web_date_format: str = WEB_DATE_FORMAT
    ) -> Optional[datetime]:
        """Convertit une date en français extraite d'une page web en un objet datetime.

        Corrige les abréviations non standard pour certains mois (exemple : "fév." devient "févr.", "juill." devient "juil.").

        Args:
            web_date (str): La chaîne représentant la date en français.
            web_date_format (str, optional): Le format de la date utilisé par la page web. Defaults to WEB_DATE_FORMAT.

        Returns:
            Optional[datetime]: L'objet datetime si la conversion réussit, sinon None.
        """
        locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

        def corrige_date(date_str: str) -> str:
            """Corrige les abréviations non standard dans la chaîne de date.

            Args:
                date_str (str): La chaîne de date originale.

            Returns:
                str: La chaîne de date corrigée.
            """
            month_replacements = {
                "fév.": "févr.",
                "juill.": "juil.",
            }
            for fr_month, fr_month_norm in month_replacements.items():
                date_str = date_str.replace(fr_month, fr_month_norm)
            return date_str

        try:
            dt: datetime = datetime.strptime(corrige_date(web_date), web_date_format)
            return dt
        except ValueError as e:
            print(f"Erreur de conversion pour la date '{web_date}': {e}")
            return None

    @staticmethod
    def get_audio_url(url: str) -> Optional[str]:
        """Récupère l'URL du fichier audio (.m4a ou .mp3) à partir de la page d'un épisode.

        Recherche dans une balise <script> contenant la clé "contentUrl".

        Args:
            url (str): L'URL de la page de l'épisode.

        Returns:
            Optional[str]: L'URL du fichier audio si trouvée, sinon None.
        """
        try:
            response: requests.Response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Erreur lors de la requête HTTP: {e}")
            return None

        soup: BeautifulSoup = BeautifulSoup(response.content, "html.parser")
        script_tag = soup.find("script", string=lambda t: t and "contentUrl" in t)

        if script_tag:
            try:
                json_text: str = script_tag.string  # type: ignore
                json_data: Dict[str, Any] = json.loads(json_text)
                audio_url: Optional[str] = None
                for item in json_data.get("@graph", []):
                    if item.get("@type") == "RadioEpisode":
                        main_entity: Dict[str, Any] = item.get("mainEntity", {})
                        audio_url = main_entity.get("contentUrl")
                        break
                return audio_url
            except (json.JSONDecodeError, KeyError, TypeError) as e:
                print(f"Erreur lors de l'analyse du JSON: {e}")
                return None

        print("Balise <script> contenant 'contentUrl' non trouvée")
        return None

    @classmethod
    def from_webpage_entry(cls, dict_web_episode: Dict[str, Any]) -> "WEB_episode":
        """Crée une instance de WEB_episode à partir d'un dictionnaire représentant une entrée de page web.

        Le dictionnaire doit contenir les clés : 'title', 'url', 'description', 'date', 'duration'.
        La variable DATE_FORMAT et la méthode get_string_from_date doivent être définies ailleurs dans le code.

        Args:
            dict_web_episode (Dict[str, Any]): Dictionnaire contenant les informations de l'épisode.

        Returns:
            WEB_episode: Une instance de WEB_episode initialisée avec les données fournies.
        """
        date_web: Optional[datetime] = cls.parse_web_date(dict_web_episode["date"])
        date_web_str: str = cls.get_string_from_date(
            date_web, DATE_FORMAT
        )  # DATE_FORMAT doit être défini en amont
        inst: WEB_episode = cls(
            date=date_web_str,
            titre=dict_web_episode["title"],
        )
        inst.description = dict_web_episode["description"]
        inst.type = "livres"
        inst.url_telechargement = cls.get_audio_url(dict_web_episode["url"])
        inst.duree = cls.get_duree_in_seconds(dict_web_episode["duration"])
        return inst

    @staticmethod
    def get_duree_in_seconds(duree: str) -> int:
        """Convertit une durée exprimée en minutes ("MM min") en secondes.

        Args:
            duree (str): La durée sous forme de chaîne.

        Returns:
            int: La durée convertie en secondes. Retourne 0 si le format n'est pas correct.
        """
        parts = duree.split(" ")
        if len(parts) == 2:
            return int(parts[0]) * 60
        return 0

In [ ]:
from web import WebPage

legacy_episodes = WebPage()
legacy_episodes[-1]

{'title': 'Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958',
 'url': 'https://www.radiofrance.fr/franceinter/podcasts/le-masque-et-la-plume/francis-walder-saint-germain-ou-la-negociation-2878243',
 'description': 'En\n 1958, les jurés du Prix Goncourt récompensent un écrivain belge, \nFrancis Walder, pour son roman historique "Saint-Germain et la \nnégociation". Les critiques du "Masque et la plume" ont du mal à cacher \nleur déception.',
 'date': '18 déc. 1958',
 'duration': '6 min'}

In [ ]:
vieil_episode = WEB_episode.from_webpage_entry(legacy_episodes[-1])

In [ ]:
vieil_episode

_oid: 678ccc978ae61760f3ab13b0
Date: 18 déc. 1958 00:00
Titre: Le Masque et les Goncourt : "Saint-Germain ou la Négociation" de Francis Walder, 1958
Description: En
 1958, les jurés du Prix Goncourt récompensent un écrivain belge, 
Francis Walder, pour son roman historique "Saint-Germain et la 
négociation". Les critiques du "Masque et la plume" ont du mal à cacher 
leur déception.
URL de téléchargement: https://media.radiofrance-podcast.net/podcast09/14007-18.12.1958-ITEMA_23787915-2024F4007E0108-27.m4a
Fichier audio: 1958/14007-18.12.1958-ITEMA_23787915-2024F4007E0108-27.m4a
Duree: 360 en secondes (00:06:00)
Transcription:  France Inter il est évident que nous l'avons profondément mûri et que nous avons lu avec beaucoup d...

In [ ]:
vieil_episode.keep()

Episode du 18 déc. 1958 00:00 deja existant


0

# Episodes entity

In [ ]:
# |export

from typing import Any, Iterator


class Episodes:
    """Classe pour rechercher et gérer la qualité des données des épisodes.

    Cette classe permet par exemple de récupérer de nouvelles transcriptions
    en se connectant à la base de données MongoDB.
    """

    def __init__(self, collection_name: str = "episodes") -> None:
        """Initialise une instance du gestionnaire d'épisodes.

        Se connecte à la base de données et charge les épisodes.

        Args:
            collection_name (str): Nom de la collection à utiliser. Par défaut "episodes".
        """
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name=collection_name
        )
        # je ne charge plus par defaut tous les episodes c'est inefficace
        self.oid_episodes = []

    # def _load_all_episodes(self) -> List["Episode"]:
    #     """Charge tous les épisodes depuis la base de données.

    #     Returns:
    #         List[Episode]: Liste des épisodes chargés.
    #     """
    #     return self.get_entries()

    def get_entries(self, request: Any = "", limit: int = -1):
        """
        Mets dans self.oid_episodes les oids correspondant à une requête spécifique, triés par date décroissante.
        Si limit est spécifié, seuls les limit premiers résultats sont conservés.
        Args:
            request (Any): Requête MongoDB à exécuter. Exemples:
                {"$or": [{"transcription": ""}, {"transcription": None}]}.
                Par défaut, une requête vide qui retourne tous les épisodes.
        """
        if limit == -1:
            results = self.collection.find(request, {"_id": 1}).sort({"date": -1})
        else:
            results = (
                self.collection.find(request, {"_id": 1})
                .sort({"date": -1})
                .limit(limit)
            )
        self.oid_episodes = [document["_id"] for document in results]

    def len_total_entries(self) -> int:
        """
        Retourne le nombre total d'épisodes dans la collection."""
        return self.collection.estimated_document_count()

    def get_missing_transcriptions(self):
        """
        Mets dans self.oid_episodes les oids correspondant aux épisodes sans transcription.
        """
        self.get_entries({"$or": [{"transcription": ""}, {"transcription": None}]})

    def get_transcriptions(self):
        """
        Mets dans self.oid_episodes les oids correspondant aux épisodes qui possèdent une transcription.
        """
        self.get_entries(
            {"$and": [{"transcription": {"$ne": None}}, {"transcription": {"$ne": ""}}]}
        )

    def __getitem__(self, index: int) -> "Episode":
        """Permet l'accès aux épisodes par indexation.

        Args:
            index (int): Position de l'épisode dans la liste.

        Returns:
            Episode: L'épisode à la position donnée.
        """
        return Episode.from_oid(self.oid_episodes[index])

    def __len__(self) -> int:
        """Retourne le nombre total d'épisodes dans oid_episodes.

        Returns:
            int: Nombre d'épisodes.
        """
        return len(self.oid_episodes)

    def __iter__(self) -> Iterator["Episode"]:
        """Permet d'itérer sur les épisodes.

        Returns:
            Iterator[Episode]: Itérateur sur la liste des épisodes.
        """
        return iter(self.oid_episodes)

    # def __str__(self) -> str:
    #     """Retourne une représentation textuelle de l'objet Episodes.

    #     La représentation inclut le nombre total d'entrées et le nombre d'épisodes sans transcription.

    #     Returns:
    #         str: Chaîne de caractères décrivant l'objet Episodes.
    #     """
    #     return (
    #         f"{self.collection.count_documents({})} entries\n"
    #         f"{len(self.get_missing_transcriptions())} missing transcriptions"
    #     )

    # def __repr__(self) -> str:
    #     """Retourne la représentation officielle de l'objet Episodes.

    #     Returns:
    #         str: Représentation de l'objet Episodes identique à celle retournée par __str__.
    #     """
    #     return self.__str__()

In [ ]:
episodes = Episodes()
episodes.get_missing_transcriptions()
len(episodes)

0

In [ ]:
episodes.len_total_entries()

206

In [ ]:
episodes.get_entries(limit=5)
len(episodes)
episodes[0]

_oid: 679649e267b093aaae847524
Date: 26 janv. 2025 10:59
Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
URL de téléchargement: https://rf.proxycast.org/f421bbf9-5e6e-4411-85a6-7c6a318d2073/14007-

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper episodes.ipynb", ".")